In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from fastcore.basics import store_attr
from fasterai.sparse.all import *
from fasterai.prune.all import *

In [ ]:
a = nn.ConvTranspose2d(3,3,3)

In [ ]:
class EdgeTrunk(nn.Module):
    def __init__(self, n_feat=64, n_block=10):
        super().__init__()

        self.filter = nn.ModuleList(
            [nn.Conv2d(n_feat, n_feat, 3, 1, 1, bias=False) for _ in range(n_block)]
        )

        self.upscale = nn.ConvTranspose2d(n_feat, n_feat, 4, 2, 1, bias=False)

    def forward(self, x):
        for filt in self.filter:
            x = F.relu(filt(x), inplace=False)
        x = self.upscale(x)

        return x


class SRNet(nn.Module):
    def __init__(self, n_channel=1, n_feat=64, n_block=5):
        super().__init__()

        self.head_in = nn.Conv2d(n_channel, n_feat, 3, 1, 1, bias=False)

        self.edge_x2 = EdgeTrunk(n_feat, n_block)
        self.tail_x2 = nn.Conv2d(n_feat, n_channel, 3, 1, 1, bias=False)

        self.edge_x4 = EdgeTrunk(n_feat, n_block)
        #self.tail_x4 = nn.Conv2d(n_feat, n_channel, 3, 1, 1, bias=False)

        self.body_x2 = nn.ConvTranspose2d(1, 1, 4, 2, 1, bias=False)
        #self.body_x4 = nn.ConvTranspose2d(1, 1, 4, 2, 1, bias=False)


    def forward(self, din):

        print(f"Input shape to SRNet: {din.shape}")  # Print the shape of input
        inp_ref = self.head_in(din)
        print(f"Output shape after head_in: {inp_ref.shape}")

        edge_x2 = self.edge_x2(inp_ref)
        print(f"Output shape after edge_x2: {edge_x2.shape}")
        edge_x4 = self.edge_x4(edge_x2)

        body_x2 = self.body_x2(din)
        eout_x2 = self.tail_x2(edge_x2)
        dout_x2 = body_x2 + eout_x2

        #body_x4 = self.body_x4(dout_x2)
        #eout_x4 = self.tail_x4(edge_x4)
        #dout_x4 = body_x4 + eout_x4
        #print(f"Final output shape from SRNet: {dout_x4.shape}")
        return eout_x2, edge_x4, dout_x2#, edge_x2 , dout_x2, eout_x4, eout_x2

In [ ]:
class EdgeTrunk(nn.Module):
    def __init__(self, n_feat=64, n_block=10):
        super().__init__()

        self.filter = nn.ModuleList(
            [nn.Conv2d(n_feat, n_feat, 3, 1, 1, bias=False) for _ in range(n_block)]
        )

        self.upscale = nn.ConvTranspose2d(n_feat, n_feat, 4, 2, 1, bias=False)

    def forward(self, x):
        for filt in self.filter:
            x = F.relu(filt(x), inplace=False)
        x = self.upscale(x)

        return x


class SRNet(nn.Module):
    def __init__(self, n_channel=1, n_feat=64, n_block=5):
        super().__init__()

        self.head_in = nn.Conv2d(n_channel, n_feat, 3, 1, 1, bias=False)

        self.edge_x2 = EdgeTrunk(n_feat, n_block)
        self.tail_x2 = nn.Conv2d(n_feat, n_channel, 3, 1, 1, bias=False)

        self.edge_x4 = EdgeTrunk(n_feat, n_block)
        self.tail_x4 = nn.Conv2d(n_feat, n_channel, 3, 1, 1, bias=False)

        self.body_x2 = nn.ConvTranspose2d(1, 1, 4, 2, 1, bias=False)
        self.body_x4 = nn.ConvTranspose2d(1, 1, 4, 2, 1, bias=False)


    def forward(self, din):

        print(f"Input shape to SRNet: {din.shape}")  # Print the shape of input
        inp_ref = self.head_in(din)
        print(f"Output shape after head_in: {inp_ref.shape}")

        edge_x2 = self.edge_x2(inp_ref)
        print(f"Output shape after edge_x2: {edge_x2.shape}")
        edge_x4 = self.edge_x4(edge_x2)

        body_x2 = self.body_x2(din)
        eout_x2 = self.tail_x2(edge_x2)
        dout_x2 = body_x2 + eout_x2

        body_x4 = self.body_x4(dout_x2)
        eout_x4 = self.tail_x4(edge_x4)
        dout_x4 = body_x4 + eout_x4
        print(f"Final output shape from SRNet: {dout_x4.shape}")
        #return dout_x4, eout_x4#, edge_x2 , dout_x2, eout_x4, eout_x2
        return dout_x4 , dout_x2, eout_x4, eout_x2

In [ ]:
class EdgeTrunk(nn.Module):
    def __init__(self, n_feat=64, n_block=10):
        super().__init__()

        self.filter = nn.ModuleList(
            [nn.Conv2d(n_feat, n_feat, 3, 1, 1, bias=False) for _ in range(n_block)]
        )

        self.upscale = nn.ConvTranspose2d(n_feat, n_feat, 4, 2, 1, bias=False)

    def forward(self, x):
        for filt in self.filter:
            x = F.relu(filt(x), inplace=False)
        x = self.upscale(x)

        return x


class SRNet(nn.Module):
    def __init__(self, n_channel=1, n_feat=64, n_block=5):
        super().__init__()

        self.head_in = nn.Conv2d(n_channel, n_feat, 3, 1, 1, bias=False)

        self.edge_x2 = EdgeTrunk(n_feat, n_block)
        self.tail_x2 = nn.Conv2d(n_feat, n_channel, 3, 1, 1, bias=False)

        self.edge_x4 = EdgeTrunk(n_feat, n_block)
        self.tail_x4 = nn.Conv2d(n_feat, n_channel, 3, 1, 1, bias=False)

        self.body_x2 = nn.ConvTranspose2d(1, 1, 4, 2, 1, bias=False)
        self.body_x4 = nn.ConvTranspose2d(1, 1, 4, 2, 1, bias=False)


    def forward(self, din):

        print(f"Input shape to SRNet: {din.shape}")  # Print the shape of input
        inp_ref = self.head_in(din)
        print(f"Output shape after head_in: {inp_ref.shape}")

        edge_x2 = self.edge_x2(inp_ref)
        print(f"Output shape after edge_x2: {edge_x2.shape}")
        edge_x4 = self.edge_x4(edge_x2)

        body_x2 = self.body_x2(din)
        eout_x2 = self.tail_x2(edge_x2)
        dout_x2 = body_x2 + eout_x2

        body_x4 = self.body_x4(dout_x2)
        eout_x4 = self.tail_x4(edge_x4)
        dout_x4 = body_x4 + eout_x4
        print(f"Final output shape from SRNet: {dout_x4.shape}")
        return dout_x4 , dout_x2, eout_x4, eout_x2

In [ ]:
model = SRNet()

In [ ]:
pruner = Pruner(model=model, context="global", criteria=small_final, example_inputs=torch.randn(1, 1, 32, 32), layer_type=nn.Conv2d)

pruner.prune_model(30)

Input shape to SRNet: torch.Size([1, 32, 32])
Output shape after head_in: torch.Size([64, 32, 32])
Output shape after edge_x2: torch.Size([64, 64, 64])
Final output shape from SRNet: torch.Size([1, 128, 128])


NotImplementedError: Unsupported module type

In [ ]:
import numpy as np

In [ ]:
%debug

> /home/HubensN/fasterai/nbs/fasterai/core/granularity.py(28)get_dim()
     26             if isinstance(m, k):
     27                 return cls._granularities[k][g]
---> 28         raise NotImplementedError("Unsupported module type")
     29 
     30     @classmethod

ipdb> m
ConvTranspose2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
ipdb> quit()


In [ ]:
import torch_pruning as tp
from torch_pruning.pruner import function

In [ ]:
model = SRNet()

In [ ]:
import torchvision

In [ ]:
model = torchvision.models.resnet18()

In [ ]:
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1, 1, 32, 32))

Input shape to SRNet: torch.Size([1, 32, 32])
Output shape after head_in: torch.Size([64, 32, 32])
Output shape after edge_x2: torch.Size([64, 64, 64])
Final output shape from SRNet: torch.Size([1, 128, 128])


<torch_pruning.dependency.DependencyGraph>

In [ ]:
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [ ]:
for m in model.modules():
    if hasattr(m, 'weight'):
        if isinstance(m, nn.Conv2d):
            #print(m)
            print(DG.get_out_channels(m))
#        setattr(m, '_init_out_channels', self.DG.get_out_channels(m))

64


KeyError: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [ ]:
%debug

> /home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch_pruning/dependency.py(533)get_out_channels()
    531         else:
    532             module = module_or_node
--> 533             pruning_dim = self.module2node[module].pruning_dim
    534         p = self.get_pruner_of_module(module)
    535         p.pruning_dim = pruning_dim

ipdb> module
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
ipdb> self.module2node
{_ElementWiseOp_0(AddBackward0): <Node: (_ElementWiseOp_0(AddBackward0))>, ConvTranspose2d(1, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False): <Node: (body_x4 (ConvTranspose2d(1, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)))>, Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False): <Node: (tail_x4 (Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)))>, _ElementWiseOp_1(ConvolutionBackward0): <Node: (_ElementWiseOp_1(ConvolutionBackward0))>, 

In [ ]:
def group_importance(group):
        handler_map = {
            function.prune_conv_out_channels: ('filter', (1, 2, 3)),
            function.prune_linear_out_channels: ('row', None),
            function.prune_conv_in_channels: ('shared_kernel', (0, 2, 3)),
            function.prune_linear_out_channels: ('column', None)
        }
        
        group_importance = []
        for dep,_ in group:
            if dep.handler in handler_map:
                for granularity, squeeze_dims in [handler_map.get(dep.handler)]:
                    if isinstance(dep.target.module, nn.Conv2d): 
                        group_importance.append(large_final(dep.target.module, granularity).squeeze(squeeze_dims))

        return torch.stack(group_importance).mean(0)

In [ ]:
for ix, grp in enumerate(DG.get_all_groups(root_module_types=[nn.Conv2d])):
    #print(grp)
    print(group_importance(grp))

tensor([0.0215])
tensor([0.1577, 0.1589, 0.1851, 0.1699, 0.1711, 0.1946, 0.1950, 0.1528, 0.2012,
        0.0936, 0.1671, 0.1755, 0.1809, 0.1209, 0.2134, 0.1907, 0.1419, 0.1998,
        0.1224, 0.2104, 0.2102, 0.1903, 0.1464, 0.1237, 0.1904, 0.1596, 0.1174,
        0.2071, 0.1591, 0.1585, 0.1297, 0.1945, 0.1433, 0.1829, 0.1825, 0.1288,
        0.1049, 0.1670, 0.1339, 0.1168, 0.1890, 0.2042, 0.1353, 0.1756, 0.1621,
        0.1986, 0.1145, 0.1274, 0.1926, 0.1321, 0.1376, 0.1953, 0.1881, 0.1340,
        0.1294, 0.1273, 0.1645, 0.1397, 0.2407, 0.2426, 0.1304, 0.2365, 0.1710,
        0.1865])
tensor([0.0219])


In [ ]:
class Pruner():
    def __init__(self, model, context, criteria, layer_type=nn.Conv2d, example_inputs=torch.randn(1,3,224,224)):
        store_attr()
        self.DG = tp.DependencyGraph()
        self.DG.build_dependency(self.model, example_inputs=example_inputs.to(next(model.parameters()).device))
        self._save_init_state()
        self._reset_threshold()
        self.init_num_groups = None

    def compute_threshold(self, sparsity):
        self.global_importance = {}
        for ix, grp in enumerate(self.DG.get_all_groups(root_module_types=[self.layer_type])):
            imp = self.group_importance(grp)
            self.global_importance[ix] = imp

        global_imp = torch.cat(list(self.global_importance.values()), dim=0)

        self.init_num_groups = self.init_num_groups or len(global_imp)
        n_pruned = np.clip(int((1-sparsity/100)*self.init_num_groups), 1, len(global_imp))
        self.global_threshold = torch.topk(global_imp, n_pruned)[0].min()

    def prune_group(self, group, ix, sparsity, round_to):
        module = group[0][0].target.module
        pruning_fn = group[0][0].handler
        pruning_idxs = self.prune_method(group, ix, sparsity, round_to)
        group = self.DG.get_pruning_group(module, pruning_fn, pruning_idxs.tolist())
        group.prune()
    
    def prune_model(self, sparsity, round_to=None):
        if self.context=='global': self.compute_threshold(sparsity)

        for ix, group in enumerate(self.DG.get_all_groups(root_module_types=[self.layer_type])):
            self.prune_group(group, ix, sparsity, round_to)

    def prune_method(self, group, ix, sparsity, round_to):
        if self.context=='global':
            imp = self.global_importance[ix]
            n_pruned = max(1, int(imp.ge(self.global_threshold).sum()))
        else:
            imp = self.group_importance(group)
            n_pruned = max(1, int((1-sparsity/100)*group[0].dep.target.module._init_out_channels))
 
        threshold = torch.topk(imp, int(self._rounded_sparsity(torch.tensor(n_pruned), round_to)))[0].min() if round_to else torch.topk(imp, n_pruned)[0].min()
        return imp.lt(threshold).nonzero().view(-1)
                
    def updated_sparsity(self, m, sparsity):
        init_channels = m._init_out_channels
        return sparsity
                
    def _save_init_state(self):
        for m in self.model.modules():
            if hasattr(m, 'weight'):
                setattr(m, '_init_out_channels', self.DG.get_out_channels(m))

    def _rounded_sparsity(self, n_to_prune, round_to):
        return max(round_to*torch.floor(n_to_prune/round_to), round_to)
    
    def _reset_threshold(self):
        self.global_threshold=None
    
    def group_importance1(self, group):
        handler_map = {
            function.prune_conv_out_channels: ('filter', (1, 2, 3)),
            function.prune_linear_out_channels: ('row', None),
            function.prune_conv_in_channels: ('shared_kernel', (0, 2, 3)),
            function.prune_linear_out_channels: ('column', None)
        }

        group_importance = [
            self.criteria(dep.target.module, granularity).squeeze(squeeze_dims)
            for dep, _ in group
            if dep.handler in handler_map
            for granularity, squeeze_dims in [handler_map.get(dep.handler)]
        ]

        return torch.stack(group_importance).mean(0)
    
    def group_importance(self, group):
            handler_map = {
                function.prune_conv_out_channels: ('filter', (1, 2, 3)),
                function.prune_linear_out_channels: ('row', None),
                function.prune_conv_in_channels: ('shared_kernel', (0, 2, 3)),
                function.prune_linear_out_channels: ('column', None)
            }

            group_importance = []
            for dep,_ in group:
                if dep.handler in handler_map:
                    for granularity, squeeze_dims in [handler_map.get(dep.handler)]:
                        #if isinstance(dep.target.module, nn.Conv2d): 
                            group_importance.append(large_final(dep.target.module, granularity).squeeze(squeeze_dims))

            return torch.stack(group_importance).mean(0)